In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
db_user = 'postgres'
db_password = '123456'
db_host = 'localhost'
db_port = '5432'
db_name = 'nordic_co2'

engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

## 2021

In [ ]:
query_2021 = '''
WITH total_2021_summary AS (

    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2) AS total_intensity
    FROM denmark.year_2021_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM finland.year_2021_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM iceland.year_2021_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM norway.year_2021_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM sweden.year_2021_hourly
    GROUP BY country

)

SELECT * FROM total_2021_summary;
'''

df_2021 = pd.read_sql_query(query_2021, engine)

display(df_2021)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_2021, x='country', y='total_intensity', s=100, color='blue')

plt.title('Intensidade de Carbono LCA por País - 2021', fontsize=14)
plt.xlabel('País')
plt.ylabel('Intensidade Total (LCA)')
plt.grid(True)
plt.tight_layout()
plt.show()



## 2022

In [ ]:
query_2022 = '''
WITH total_2022_summary AS (

    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2) AS total_intensity
    FROM denmark.year_2022_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM finland.year_2022_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM iceland.year_2022_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM norway.year_2022_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM sweden.year_2022_hourly
    GROUP BY country

)

SELECT * FROM total_2022_summary;
'''

df_2022 = pd.read_sql_query(query_2022, engine)

display(df_2022)

## 2023

In [ ]:
query_2023 = '''
WITH total_2023_summary AS (

    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2) AS total_intensity
    FROM denmark.year_2023_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM finland.year_2023_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM iceland.year_2023_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM norway.year_2023_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM sweden.year_2023_hourly
    GROUP BY country

)

SELECT * FROM total_2023_summary;
'''

df_2023 = pd.read_sql_query(query_2023, engine)
display(df_2023)


## 2024

In [ ]:
query_2024 = '''
WITH total_2024_summary AS (

    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2) AS total_intensity
    FROM denmark.year_2024_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM finland.year_2024_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM iceland.year_2024_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM norway.year_2024_hourly
    GROUP BY country

    UNION ALL
    
    SELECT country, ROUND(CAST(SUM(carbon_intensity_lca) AS numeric), 2)
    FROM sweden.year_2024_hourly
    GROUP BY country

)

SELECT * FROM total_2024_summary;
'''

df_2024 = pd.read_sql_query(query_2024, engine)
display(df_2024)

## Merging all DataFrames

In [ ]:
df_2021['year'] = 2021
df_2022['year'] = 2022
df_2023['year'] = 2023
df_2024['year'] = 2024

df_all_years = pd.concat([df_2021, df_2022, df_2023, df_2024], ignore_index=True)

df_all_years = df_all_years[['country', 'year', 'total_intensity']]

display(df_all_years)